#Version  del model

In [18]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [19]:
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close()
df        

,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2
...,...,...,...,...,...,...,...,...
48775,2021-2022,2,42,5/29/22,None,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,5/29/22,None,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,5/29/22,None,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,5/29/22,None,CD Tenerife,FC Cartagena,None


In [20]:
df["result"] = None
df["goals_home"] = df["score"].str.split(":").str[0]
df["goals_visitor"] = df["score"].str.split(":").str[1]
df.loc[df["goals_home"] > df["goals_visitor"], "result"] = 'home'
df.loc[df["goals_home"] == df["goals_visitor"], "result"] = 'tie'
df.loc[df["goals_home"] < df["goals_visitor"], "result"] = 'visitor'
df

,season,division,matchday,date,time,home_team,away_team,score,result,goals_home,goals_visitor
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3,visitor,2,3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2,home,3,2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0,home,5,0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1,tie,1,1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2,visitor,0,2
...,...,...,...,...,...,...,...,...,...,...,...
48775,2021-2022,2,42,5/29/22,None,Real Oviedo,UD Ibiza,None,None,None,None
48776,2021-2022,2,42,5/29/22,None,Real Sociedad B,Real Zaragoza,None,None,None,None
48777,2021-2022,2,42,5/29/22,None,Sporting Gijón,UD Las Palmas,None,None,None,None
48778,2021-2022,2,42,5/29/22,None,CD Tenerife,FC Cartagena,None,None,None,None


In [4]:
df_ranking = df
df_ranking["goals_home"] = df_ranking["goals_home"].fillna(0).astype(int)
df_ranking["goals_visitor"] = df_ranking["goals_visitor"].fillna(0).astype(int)

points_home = df_ranking[df_ranking["result"] == "home"].groupby(["season", "division", "home_team", "matchday"])["result"].count() * 3
points_home.index = points_home.index.set_names("team", level="home_team")
points_tie1 = df_ranking[df_ranking["result"] == "tie"].groupby(["season", "division", "home_team", "matchday"])["result"].count()
points_tie1.index = points_tie1.index.set_names("team", level="home_team")
points_tie2 = df_ranking[df_ranking["result"] == "tie"].groupby(["season", "division", "away_team", "matchday"])["result"].count()
points_tie2.index = points_tie2.index.set_names("team", level="away_team")
points_visitor = df_ranking[df_ranking["result"] == "visitor"].groupby(["season", "division", "away_team", "matchday"])["result"].count() * 3
points_visitor.index = points_visitor.index.set_names("team", level="away_team")
points0_home = df_ranking[df_ranking["result"] == "visitor"].groupby(["season", "division", "home_team", "matchday"])["result"].count() * 0
points0_home.index = points0_home.index.set_names("team", level="home_team")
points0_visitor = df_ranking[df_ranking["result"] == "home"].groupby(["season", "division", "away_team", "matchday"])["result"].count() * 0
points0_visitor.index = points0_visitor.index.set_names("team", level="away_team")

points = points_home.add(points_tie1, fill_value=0).add(points_tie2, fill_value=0).add(points_visitor, fill_value=0).add(points0_home, fill_value=0).add(points0_visitor, fill_value=0).astype(int)

loss_home = df_ranking[df_ranking["result"] == "visitor"].groupby(["season", "division", "home_team", "matchday"])["result"].count()
loss_home.index = loss_home.index.set_names("team", level="home_team")
loss_visitor = df_ranking[df_ranking["result"] == "home"].groupby(["season", "division", "away_team", "matchday"])["result"].count()
loss_visitor.index = loss_visitor.index.set_names("team", level="away_team")
loss = loss_home.add(loss_visitor, fill_value=0).astype(int)

win_home = df_ranking[df_ranking["result"] == "home"].groupby(["season", "division", "home_team", "matchday"])["result"].count()
win_home.index = win_home.index.set_names("team", level="home_team")
win_visitor = df_ranking[df_ranking["result"] == "visitor"].groupby(["season", "division", "away_team", "matchday"])["result"].count()
win_visitor.index = win_visitor.index.set_names("team", level="away_team")
win = win_home.add(win_visitor, fill_value=0).astype(int)

tie_home = df_ranking[df_ranking["result"] == "tie"].groupby(["season", "division", "home_team", "matchday"])["result"].count()
tie_home.index = tie_home.index.set_names("team", level="home_team")
tie_visitor = df_ranking[df_ranking["result"] == "tie"].groupby(["season", "division", "away_team", "matchday"])["result"].count()
tie_visitor.index = tie_visitor.index.set_names("team", level="away_team")
tie = tie_home.add(tie_visitor, fill_value=0)

goals_home = df_ranking.groupby(["season", "division", "home_team", "matchday"])["goals_home"].sum()
goals_home.index = goals_home.index.set_names("team", level="home_team")
goals_visitor = df_ranking.groupby(["season", "division", "away_team", "matchday"])["goals_visitor"].sum()
goals_visitor.index = goals_visitor.index.set_names("team", level="away_team")
GF = goals_home.add(goals_visitor, fill_value=0)

goals_home = df_ranking.groupby(["season", "division", "home_team", "matchday"])["goals_visitor"].sum()
goals_home.index = goals_home.index.set_names("team", level="home_team")
goals_visitor = df_ranking.groupby(["season", "division", "away_team", "matchday"])["goals_home"].sum()
goals_visitor.index = goals_visitor.index.set_names("team", level="away_team")
GA = goals_home.add(goals_visitor, fill_value=0)

GD = GF - GA

points = points.groupby(["season", "division", "team"]).cumsum()
GD = GD.groupby(["season", "division", "team"]).cumsum()
GF = GF.groupby(["season", "division", "team"]).cumsum()
GA = GA.groupby(["season", "division", "team"]).cumsum()

win2 = win.add(loss * 0, fill_value=0).add(tie * 0, fill_value=0).groupby(["season", "division", "team"]).cumsum().astype(int)
tie2 = tie.add(loss * 0, fill_value=0).add(win * 0, fill_value=0).groupby(["season", "division", "team"]).cumsum().astype(int)
loss2 = loss.add(tie * 0, fill_value=0).add(win * 0, fill_value=0).groupby(["season", "division", "team"]).cumsum().astype(int)

In [17]:
df_teams = pd.DataFrame({
    "GF": GF,
    "GA": GA,
    "GD": GD,
    "W": win2,
    "L": loss2,
    "T": tie2,
    "Pts": points
}).reset_index()

def add_ranking(df):
    df.insert(2, "rank", df["Pts"].rank(ascending=False, method="first"))
    return df
df_teams = df_teams.reset_index(drop=True).sort_values(["Pts", "GD", "GF"], ascending=[False, False, False]).reset_index(drop=True)
df_teams = df_teams.groupby(["season", "division", "matchday"]).apply(add_ranking).reset_index(drop=True).sort_values(["season", "division", "matchday"], ascending=[False, True, True]).reset_index(drop=True)

conn = sqlite3.connect('clasification.sqlite')
df_teams.to_sql('clasification', conn, index=False)
conn.close()

df_teams = df_teams[df_teams["season"] == "2000-2001"]

In [6]:
df["prediction"] = np.where(df["result"] == "home", 1, 
                        np.where(df["result"] == "visitor", 2, 
                                 np.where(df["result"] == "tie", "X", None)))
df_games=df.dropna(subset=["score"])
df_games = df_games[df_games["season"] == "2000-2001"]

In [7]:
df_combo = pd.merge(df_games, df_teams, left_on='home_team', right_on='team', how='left', suffixes=("_home", ""))
df_combo = pd.merge(df_combo, df_teams, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_combo = df_combo.drop_duplicates()

In [8]:
from sklearn.preprocessing import LabelEncoder

# Supongamos que tienes un DataFrame df con una columna 'etiqueta' de tipo 'object'
le = LabelEncoder()
df_combo['prediction'] = le.fit_transform(df_combo['result'])


In [9]:
X = df_combo[['rank', 'GD', "W", "Pts"]]
y = df_combo['prediction']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Crear y entrenar un modelo de clasificación (por ejemplo, Random Forest)
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [13]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

Precisión del modelo: 0.5041248377563816


In [14]:
# Mostrar un informe de clasificación
print(classification_report(y_test, y_pred, target_names=['home_win(1)', 'visitor_win(2)', 'tie(X)']))

                precision    recall  f1-score   support

   home_win(1)       0.54      0.75      0.63    125730
visitor_win(2)       0.45      0.36      0.40     82998
        tie(X)       0.43      0.21      0.29     64010

      accuracy                           0.50    272738
     macro avg       0.47      0.44      0.44    272738
  weighted avg       0.49      0.50      0.48    272738



In [15]:
# Supongamos que tienes un modelo ya entrenado llamado 'model'

# Codificar los nombres de los equipos
home_team = 'Real Madrid'  # Reemplaza con el nombre del equipo local
away_team = 'Barcelona'  # Reemplaza con el nombre del equipo visitante

# Proporcionar las características al modelo (en este ejemplo, se asumen 2 goles para el equipo local y 1 gol para el equipo visitante)
caracteristicas = [[]]

# Realizar la predicción
resultado_predicho = model.predict(caracteristicas)

# Función para mapear los resultados
def mapear_resultado(resultado):
    if resultado == 0:
        return '1'  # Gana el equipo local
    elif resultado == 1:
        return '2'  # Gana el equipo visitante
    else:
        return 'X'  # Empate

# Decodificar el resultado
resultado = mapear_resultado(resultado_predicho)

print(f'El resultado predicho del partido es: {resultado}')


/home/la-quiniela-main/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required by RandomForestClassifier.